In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib as plt
from functools import reduce
df_features = pd.read_csv("final_features.csv")

In [ ]:
length_of_stay_outcomes = pd.read_csv("prolonged_length_of_stay_results.csv")
mortality_outcomes = pd.read_csv("hospital_mortality_results.csv")
aki_outcomes = pd.read_csv("aki_status_results.csv")

In [ ]:
aki_outcomes.head()

### add misprediction column

In [ ]:
def add_misprediction_col(df, actual_column):
  prediction_columns = list(set(df.columns.to_list()) - set([actual_column, 'uniquepid']))
  df_mispredict = df.copy()
  misprediction_columns = []
  for column in prediction_columns:
    df_mispredict[f'misprediction_{column}'] = (df[column] != df[actual_column]).astype(int)
    misprediction_columns.append(f'misprediction_{column}')
  df_mispredict[f'misprediction_count_{actual_column}'] = df_mispredict[misprediction_columns].sum(axis=1)
  df_mispredict.drop(inplace=True, columns=misprediction_columns)
  return df_mispredict

In [ ]:
add_misprediction_col(length_of_stay_outcomes, 'actual_prolonged_length_of_stay').to_csv('length_of_stay_misprediction.csv')
add_misprediction_col(mortality_outcomes, 'actual_hospital_mortality').to_csv('mortality_misprediction.csv')
add_misprediction_col(aki_outcomes, 'actual_aki_status').to_csv('aki_misprediction.csv')

In [ ]:
length_of_stay_misprediction = pd.read_csv('length_of_stay_misprediction.csv')
mortality_misprediction = pd.read_csv('mortality_misprediction.csv')
aki_misprediction = pd.read_csv('aki_misprediction.csv')

In [ ]:
aki_misprediction

merging dataframes

In [ ]:
dataframes = [length_of_stay_misprediction, mortality_misprediction, aki_misprediction]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='uniquepid', how='inner'), dataframes)
misprediction_cols = [col for col in merged_df.columns if col.startswith('misprediction_count_')]
merged_df['total_misprediction_count'] = merged_df[misprediction_cols].sum(axis=1)

In [ ]:
merged_df

In [ ]:
merged_df['total_misprediction_count'].value_counts()

In [ ]:
length_of_stay_outcomes.columns.to_list()

In [ ]:
mortality_outcomes.columns.to_list()

In [ ]:
aki_outcomes.columns.to_list()